In [2]:
from selenium import webdriver
from time import sleep
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup

options = Options()
options.headless = True
driver = webdriver.Firefox(options=options)

url = "https://www2.hm.com/en_us/men/concepts/conscious-sustainable-style.html"

driver.get(url)
sleep(3)

while True:
    try:
        driver.find_element_by_class_name("js-load-more").click()
        sleep(3)
    except:
        break

driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

soup = BeautifulSoup(driver.page_source,'html.parser')

links = []

for product in soup.find_all(class_="item-link"):
    links.append(product['href'])

driver.quit()

len(links)

127

In [3]:
url = "https://www2.hm.com/en_us/women/concepts/conscious-sustainable-style.html"

driver = webdriver.Firefox(options=options)

driver.get(url)
sleep(3)

while True:
    try:
        driver.find_element_by_class_name("js-load-more").click()
        sleep(3)
    except:
        break

driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

soup = BeautifulSoup(driver.page_source,'html.parser')

for product in soup.find_all(class_="item-link"):
    links.append(product['href'])

driver.quit()
len(links)

1060

1060 items

In [4]:
# Splitting to avoid throttling
links1 = links[0:530]
links2 = links[530:]

In [5]:
import random
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from IPython.display import clear_output

sleepTimes = [2.1, 2.8, 3.2]

info_text = []

def get_info(link_list):
    driver = webdriver.Firefox(options=options)

    for link in link_list:
        clear_output(wait=True)
        driver.get("https://www2.hm.com" + link)
        # some don't have product info    
        try:    
            element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "js-open-sustainability")))
            driver.find_element_by_class_name("js-open-sustainability").click()
        except:
            continue
        soup = BeautifulSoup(driver.page_source,'html.parser')

        # pieces are separate li, so add to a list, join into one item, then append that list to keep one product per item in info_text
        li_list = []
        for li in soup.select('ul.ProductBackground-module--sustainabilityList__15IkU > li'):
            if "%" in li.text:
                li_list.append(li.text)
            else:
                continue
        
        li_list_join = " ".join(li_list)
        info_text.append(li_list_join)
        
        print("slice -", links.index(link_list[0]), "Current progress:", round( ( (link_list.index(link) + 1) / len(link_list) )*100, 2 ), "%")
        sleep(random.choice(sleepTimes))
    
    driver.quit()

get_info(links1)
sleep(300)
get_info(links2)

print(len(info_text))

slice - 530 Current progress: 100.0 %
1019


info from 1019 products

In [6]:
import pandas as pd

materials_hm = pd.Series(info_text)
materials_hm.to_csv("materials_hm.csv") 